In [1]:
import copy
import torch
from torch.utils.data import *
from transformers import *
import inspect
import sys
sys.path.insert(0, "..")

from models import *
from logic import *
from my_datasets import *

from utils import *

import numpy as np

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
n, r = 5, 8
ap, bp, tp, sp = 0.2, 0.2, 0.4, 0.1
num_arsteps = 3

qed_train_dataset_config = OneShotQedDatasetConfig(r,n,ap,bp,tp,dataset_len=1000,seed=1234)
qed_test_dataset_config = OneShotQedDatasetConfig(r,n,ap,bp,tp,dataset_len=500,seed=2345)
qed_train_dataset = OneShotQedDataset(qed_train_dataset_config)
qed_test_dataset = OneShotQedDataset(qed_test_dataset_config)

succ_train_dataset_config = PredictSuccDatasetConfig(r,n,ap,bp,tp,dataset_len=1000,seed=1234)
succ_test_dataset_config = PredictSuccDatasetConfig(r,n,ap,bp,tp,dataset_len=500,seed=2345)
succ_train_dataset = PredictSuccDataset(succ_train_dataset_config)
succ_test_dataset = PredictSuccDataset(succ_test_dataset_config)

arsteps_train_dataset_config = AutoRegFixedStepsDatasetConfig(r,n,ap,bp,sp,num_arsteps,dataset_len=1000,seed=1234)
arsteps_test_dataset_config = AutoRegFixedStepsDatasetConfig(r,n,ap,bp,sp,num_arsteps,dataset_len=500,seed=2345)
arsteps_train_dataset = AutoRegFixedStepsDataset(arsteps_train_dataset_config)
arsteps_test_dataset = AutoRegFixedStepsDataset(arsteps_test_dataset_config)

In [3]:
mytf_qed_base = get_seqcls_model("mytf", num_labels=2, problem_type="single_label_classification")
mytf_qed = OneShotQedTaskModel(OneShotQedTaskConfig(n, mytf_qed_base))

mytf_seqcls_base = get_seqcls_model("mytf", num_labels=2, problem_type="multi_label_classification")
mytf_succ = PredictSuccTaskModel(PredictSuccTaskConfig(n, mytf_seqcls_base))
mytf_arsteps = AutoRegFixedStepsTaskModel(AutoRegFixedStepsTaskConfig(n, num_arsteps, mytf_seqcls_base))

gpt2_qed_base = get_seqcls_model("gpt2", num_labels=2, problem_type="single_label_classification")
gpt2_qed = OneShotQedTaskModel(OneShotQedTaskConfig(n, gpt2_qed_base))

gpt2_seqcls_base = get_seqcls_model("gpt2", num_labels=2, problem_type="multi_label_classification")
gpt2_succ = PredictSuccTaskModel(PredictSuccTaskConfig(n, gpt2_seqcls_base))
gpt2_arsteps = AutoRegFixedStepsTaskModel(AutoRegFixedStepsTaskConfig(n, num_arsteps, gpt2_seqcls_base))

In [4]:
qed_training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy = "epoch",
    num_train_epochs = 50,
    per_device_train_batch_size = 24,
    per_device_eval_batch_size = 24,
    logging_steps = 5
)

succ_training_args = qed_training_args
arsteps_training_args = qed_training_args

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
### QED GPT2
gpt2_qed_trainer = Trainer(gpt2_qed, qed_training_args,
    train_dataset = qed_train_dataset,
    # eval_dataset = qed_test_dataset,
    eval_dataset = qed_train_dataset,
    compute_metrics = qed_compute_metrics)
gpt2_qed_trainer.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 50
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 2,100
  Number of trainable parameters = 125,239,296
GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.654800,0.698147,0.000000,0.605500
2,0.606100,0.622546,0.000000,0.813000
3,0.475100,0.604281,0.000000,0.653500
4,0.707700,0.798369,0.000000,0.572500


GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`
GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`
GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`
GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`
GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`
GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_e

In [ ]:
### QED MyTf
mytf_qed_trainer = Trainer(mytf_qed, qed_training_args,
    train_dataset = qed_train_dataset,
    eval_dataset = qed_test_dataset,
    compute_metrics = qed_compute_metrics)
mytf_qed_trainer.train()

In [ ]:
### SUCC GPT2
gpt2_succ_trainer = Trainer(gpt2_succ, succ_training_args,
    train_dataset = succ_train_dataset,
    eval_dataset = succ_test_dataset,
    compute_metrics = succ_compute_metrics)
gpt2_succ_trainer.train()

In [ ]:
### SUCC MyTF
mytf_succ_trainer = Trainer(mytf_succ, succ_training_args,
    train_dataset = succ_train_dataset,
    eval_dataset = succ_test_dataset,
    compute_metrics = succ_compute_metrics)
mytf_succ_trainer.train()

In [ ]:
### ARSteps GPT2
gpt2_arsteps_trainer = Trainer(gpt2_arsteps_model, arsteps_training_args,
    train_dataset = arsteps_train_dataset,
    eval_dataset = arsteps_test_dataset,
    compute_metrics = arsteps_compute_metrics)
gpt2_arsteps_trainer.train()

In [ ]:
batch = next(iter(DataLoader(qed_train_dataset, batch_size=7)))
mytf_qed.cpu()
out = mytf_qed(**batch)

In [ ]:
out.logits.shape